# 13.2. MovieLens 1M Dataset

MovieLens 1M là bộ dữ liệu chứa khoảng một triệu đánh giá phim, do ~6.000 người dùng đánh giá ~4.000 phim. Dữ liệu được chia thành 3 bảng nhỏ:

ratings: các đánh giá của người dùng  
users: thông tin người dùng (tuổi, giới tính, nghề nghiệp, mã bưu điện)  
movies: thông tin phim (thể loại, năm)  
Bộ dữ liệu hữu ích cho việc phát triển hệ gợi ý. Để dùng trong Python/Jupyter, ta thường tải mỗi bảng vào pandas DataFrame bằng pandas.read_table — chạy đoạn code tương ứng trong một ô Jupyter.

In [3]:
# Import thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Đọc dữ liệu từ file users.dat
unames = ["user_id", "gender", "age", "occupation", "zip"]
users = pd.read_table("datasets/movielens/users.dat", sep="::",
header=None, names=unames, engine="python")

In [5]:
# Đọc dữ liệu từ file ratings.dat
rnames = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_table("datasets/movielens/ratings.dat", sep="::",
header=None, names=rnames, engine="python")

In [6]:
# Đọc dữ liệu từ file movies.dat
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("datasets/movielens/movies.dat", sep="::",
header=None, names=mnames, engine="python")

Để xác nhận mọi thứ đã thành công, xem từng DataFrame:

In [7]:
# Hiển thị 5 dòng đầu tiên của users
users.head(5)

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
# Hiển thị 5 dòng đầu tiên của ratings
ratings.head(5)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [9]:
# Hiển thị 5 dòng đầu tiên của movies
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
# Xem DataFrame ratings
ratings 

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


Tuổi và nghề được mã hóa bằng các số tương ứng các nhóm trong file README. Dữ liệu nằm rải rác trên 3 bảng nên khó phân tích (ví dụ: tính điểm trung bình của một phim theo giới tính và độ tuổi). Dùng pandas.merge để ghép ratings với users rồi ghép kết quả đó với movies — pandas sẽ tự chọn cột khóa dựa trên tên trùng nhau.

In [11]:
# Kết hợp ba DataFrame thành một DataFrame duy nhất
data = pd.merge(pd.merge(ratings, users), movies) 

In [12]:
# Xem DataFrame data
data

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,M,25,6,11106,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,956704887,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,956704746,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,956715648,M,25,6,11106,Sophie's Choice (1982),Drama


In [13]:
# Xem một dòng dữ liệu trong DataFrame data
data.iloc[0]

user_id                                            1
movie_id                                        1193
rating                                             5
timestamp                                  978300760
gender                                             F
age                                                1
occupation                                        10
zip                                            48067
title         One Flew Over the Cuckoo's Nest (1975)
genres                                         Drama
Name: 0, dtype: object

Để tính điểm trung bình của mỗi phim theo từng giới tính, ta có thể dùng phương thức pivot_table:

In [14]:
# Tính điểm đánh giá trung bình cho mỗi phim
mean_ratings = data.pivot_table("rating", index="title",
columns="gender", aggfunc="mean")

In [15]:
# Hiển thị 5 dòng đầu tiên của mean_ratings
mean_ratings.head(5)

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024


Kết quả là một DataFrame mới có tiêu đề phim làm chỉ mục (index) và giới tính làm nhãn cột. Để giữ lại chỉ những phim có ít nhất 250 lượt đánh giá, nhóm dữ liệu theo title rồi dùng size() để lấy số lượng đánh giá cho từng phim (một Series), sau đó lọc theo điều kiện >= 250.

In [16]:
# Tính số lượng đánh giá cho mỗi phim
ratings_by_title = data.groupby("title").size()

In [17]:
# Hiển thị 5 dòng đầu tiên của ratings_by_title
ratings_by_title.head()

title
$1,000,000 Duck (1971)            37
'Night Mother (1986)              70
'Til There Was You (1997)         52
'burbs, The (1989)               303
...And Justice for All (1979)    199
dtype: int64

In [18]:
# Lọc các phim có ít nhất 250 đánh giá
active_titles = ratings_by_title.index[ratings_by_title >= 250]

In [19]:
# Hiển thị DataFrame active_titles
active_titles

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'X-Men (2000)', 'Year of Living Dangerously (1982)',
       'Yellow Submarine (1968)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Sherlock Holmes (1985)',
       'Zero Effect (1998)', 'eXistenZ (1999)'],
      dtype='object', name='title', length=1216)

Chỉ mục các phim có ít nhất 250 đánh giá có thể dùng để chọn các hàng tương ứng trong mean_ratings bằng .loc:

In [20]:
# Lọc mean_ratings để chỉ bao gồm các phim trong active_titles
mean_ratings = mean_ratings.loc[active_titles]

In [21]:
# Hiển thị DataFrame mean_ratings
mean_ratings

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


Để xem những phim được khán giả nữ đánh giá cao nhất, ta có thể sắp xếp theo cột F theo thứ tự giảm dần:

In [ ]:
# Sắp xếp các phim theo điểm đánh giá trung bình của nữ giới
top_female_ratings = mean_ratings.sort_values("F", ascending=False)

In [ ]:
# Hiển thị 5 dòng đầu tiên của top_female_ratings
top_female_ratings.head()

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415


## Đo sự khác biệt trong đánh giá

Giả sử bạn muốn tìm những phim có sự phân hóa nhiều nhất giữa khán giả nam và nữ. Một cách là thêm cột vào mean_ratings chứa hiệu điểm trung bình giữa hai giới, rồi sắp xếp theo giá trị đó:

In [26]:
# Tính toán độ chênh lệch giữa điểm đánh giá trung bình của nam và nữ
mean_ratings["diff"] = mean_ratings["M"] - mean_ratings["F"]

Sắp xếp theo "diff" sẽ cho thấy những phim có sự khác biệt lớn nhất về điểm đánh giá, giúp ta biết phim nào được nữ giới ưa thích hơn:

In [27]:
# Sắp xếp các phim theo độ chênh lệch điểm đánh giá (nữ thích hơn nam nếu diff âm)
sorted_by_diff = mean_ratings.sort_values("diff")

In [28]:
# Hiển thị 5 dòng đầu tiên của sorted_by_diff
sorted_by_diff.head()

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777


Đảo ngược thứ tự các dòng và lấy 10 dòng đầu tiên sẽ cho ra các phim được nam giới ưa thích hơn nữ:

In [29]:
# Các phim nam giới thích hơn nữ (diff dương lớn nhất)
sorted_by_diff[::-1].head()

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787


Để tìm các phim gây nhiều tranh cãi nhất giữa các khán giả (không phân biệt giới tính), ta có thể đo bằng độ lệch chuẩn (standard deviation) của điểm đánh giá. Đầu tiên, tính độ lệch chuẩn theo từng phim, sau đó lọc các phim có ít nhất 250 đánh giá:

In [30]:
# Tính độ lệch chuẩn điểm đánh giá cho mỗi phim
rating_std_by_title = data.groupby("title")["rating"].std()

In [31]:
# Lọc độ lệch chuẩn cho các phim đang hoạt động
rating_std_by_title = rating_std_by_title.loc[active_titles]

In [32]:
# Hiển thị 5 dòng đầu tiên của rating_std_by_title
rating_std_by_title.head()

title
'burbs, The (1989)                   1.107760
10 Things I Hate About You (1999)    0.989815
101 Dalmatians (1961)                0.982103
101 Dalmatians (1996)                1.098717
12 Angry Men (1957)                  0.812731
Name: rating, dtype: float64

Sau đó, ta sắp xếp theo thứ tự giảm dần và chọn 10 dòng đầu tiên, đó là các phim gây tranh cãi nhất:

In [33]:
# Sắp xếp theo độ lệch chuẩn giảm dần và lấy 10 phim đầu tiên
rating_std_by_title.sort_values(ascending=False)[:10]

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

Bạn có thể nhận thấy rằng thể loại phim được lưu dưới dạng một chuỗi phân cách bằng dấu gạch đứng (|), vì một phim có thể thuộc nhiều thể loại. Để giúp chúng ta nhóm dữ liệu đánh giá theo thể loại, ta có thể dùng phương thức explode trên DataFrame. Hãy xem cách hoạt động của nó. Đầu tiên, ta có thể tách chuỗi thể loại thành danh sách các thể loại bằng phương thức str.split trên Series:

In [34]:
# Hiển thị 5 dòng đầu tiên của cột genres trong DataFrame movies
movies["genres"].head()

0     Animation|Children's|Comedy
1    Adventure|Children's|Fantasy
2                  Comedy|Romance
3                    Comedy|Drama
4                          Comedy
Name: genres, dtype: object

In [36]:
# Hiển thị 5 dòng đầu tiên của cột genres sau khi tách
movies["genres"].head().str.split("|")

0     [Animation, Children's, Comedy]
1    [Adventure, Children's, Fantasy]
2                   [Comedy, Romance]
3                     [Comedy, Drama]
4                            [Comedy]
Name: genres, dtype: object

In [37]:
# Hiển thị 5 dòng đầu tiên của movies
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Bây giờ, gọi movies.explode("genre") sẽ tạo ra một DataFrame mới với một dòng cho mỗi phần tử trong danh sách thể loại của từng phim. Ví dụ, nếu một phim được phân loại vừa là "Comedy" vừa là "Romance", thì kết quả sẽ có hai dòng: một dòng chỉ với "Comedy" và một dòng chỉ với "Romance".

In [43]:
# explode cột genre để tạo một dòng cho mỗi thể loại
movies_exploded = movies.explode("genres")

In [44]:
# Hiển thị 10 dòng đầu tiên của movies_exploded
movies_exploded[:10] 

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Bây giờ, chúng ta có thể gộp cả ba bảng lại với nhau và nhóm theo thể loại phim.

In [ ]:
# Tạo DataFrame ratings_with_genre bằng cách kết hợp movies_exploded, ratings và users
ratings_with_genre = pd.merge(pd.merge(movies_exploded, ratings), users)

In [47]:
# Tạo bảng pivot để tính điểm đánh giá trung bình theo thể loại và độ tuổi
genre_ratings = (ratings_with_genre.groupby(["genres", "age"])["rating"].mean().unstack("age"))

In [48]:
# Hiển thị 10 dòng đầu tiên của genre_ratings
genre_ratings[:10]

age,1,18,25,35,45,50,56
genres,,,,,,,
Action,3.393617,3.233253,3.282209,3.460765,3.442227,3.527273,3.566745
Action|Adventure,3.632035,3.636634,3.713771,3.642051,3.598513,3.734525,3.764706
Action|Adventure|Animation,4.636364,4.250000,4.059211,4.150000,3.846154,3.888889,3.800000
Action|Adventure|Animation|Children's|Fantasy,2.875000,2.588235,2.411765,2.814815,3.000000,3.333333,3.000000
Action|Adventure|Animation|Horror|Sci-Fi,3.600000,3.433962,3.620462,3.531034,3.125000,3.888889,3.250000
Action|Adventure|Children's,1.200000,1.363636,1.263158,1.571429,1.000000,1.000000,NaN
Action|Adventure|Children's|Comedy,2.593750,2.254545,2.312796,2.229167,2.441860,2.222222,2.125000
Action|Adventure|Children's|Fantasy,2.333333,1.692308,1.941176,2.375000,3.000000,4.000000,NaN
Action|Adventure|Children's|Sci-Fi,2.296296,1.926829,1.658120,1.739130,2.055556,2.437500,2.333333
